#### Step 1 :

- Import packages
- Load dataset

In [ ]:
# For data manipulation
import numpy as np
import pandas as pd

# For data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

# For data modeling
from xgboost import XGBClassifier
from xgboost import plot_importance

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# For metrics and helpful functions
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import plot_tree
from sklearn import metrics

# Future Warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# Load dataset

train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

# 1) Exploratory Data Analysis (EDA)
## a) Loading and initial exploration

In [ ]:
# Head data
train_data.head()

Columns : 
- PassengerID : Passenger ID. It's the index.
- Survived : 0 = No, 1 = Yes
- Pclass : Ticket class. 1 = 1st, 2 = 2nd, 3 = 3rd
- Name
- Sex
- Age
- SibSp: Siblings/Spouses Aboard. Mistresses and fiancés were ignored.
- Parch : Parents/Children Aboard. Some children travelled only with a nanny, therefore parch=0 for them.
- Ticket : Ticket Number
- Fare : Passenger Fare
- Cabin : Cabin number
- Embarked : Port of Embarkation. C = Cherbourg, Q = Queenstown, S = Southampton


In [ ]:
#head test
test_data.head()

Test_data shows the same columns, except survived, which is hidden. This will allow you to evaluate the model you've created.

In [ ]:
train_data.info()

The dtype of several columns will have to be reworked to be able to create a model, for example 'Sex', 'Embarked' ...

In [ ]:
train_data.describe()

There are 891 ID on the dataset.
There must be missing data in the 'Age' dataset. The average age is 29.69.
The survival rate is 38%.

In [ ]:
#check NaN value
isnull_train = train_data.isnull().sum()

isnull_test = test_data.isnull().sum()

print('Train :\n',isnull_train,'\n')
print('-'*40)
print('Test :\n',isnull_test)

In [ ]:
#Check Nan
def percentage_null(data, col) :
    value = (data[col].isnull().sum() / len(data['PassengerId']))*100
    return value
    
columns = ['Age', 'Cabin', 'Embarked']

for col in columns :
    percentage = percentage_null(train_data, col)
    print('The percentage of null values for',col, 'is :' ,percentage.round(2), '%' )

Conclusion :

- The number of missing values in the “Cabin” column is important. Depending on the data exploration, I'll see if I delete this column, or if it can still give me important information with this question: Why are they missing?

- The number of missing values for age is also important, but the correlation with survival rate should be significant. I think we should keep the column and replace the missing values.

- The 'Embarked' column has only 2 missing values, which is insignificant.

In [ ]:
#Duplicate
duplicate_train = train_data.duplicated().sum()
duplicate_test = test_data.duplicated().sum()

print('Duplicate Train :', duplicate_train)
print('Duplicate Test :', duplicate_test)

## b) Data cleaning

In [ ]:
# Cleaning col 'Age'
## Use the average of the 3 preceding and following data to replace the age by the average.

def cleaning_age(data, col, window) :
    data[col] = data[col].fillna(data[col].rolling(window=window, center=True, min_periods=1).mean())
    return data
    
train_and_test = [train_data, test_data]

for data in train_and_test :
    data = cleaning_age(data,'Age',7)
    

In [ ]:
#Cleaning Embarked
train_data['Embarked'].value_counts()

I will replace the few missing values with the most represented port.

In [ ]:
#replace Nan By S in train_data[Embarked]
train_data['Embarked'].fillna('S', inplace=True)

#replace NaN by median in test_data[Fare]
test_data['Fare'] = pd.to_numeric(test_data['Fare'], errors='coerce')
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

## c) Outliers

In [ ]:
# Set the figure and axes
fig, ax = plt.subplots(1, 2, figsize=(30,5))

sns.boxplot(x=train_data['Age'], orient='h',ax=ax[0])
ax[0].set_title("Outliers of age", fontsize='20')

sns.boxplot(x=train_data['Fare'], orient='h', ax=ax[1])
ax[1].set_title("Outliers of fare", fontsize='20')

plt.tight_layout()

There don't seem to be any outliers in the 'Age' column. Indeed, in 1912, it was quite possible to reach 80.
As for 'Fare', which averages £32, prices of up to £500 seem astonishing.
That's why I'll try to determine the number of outliers per class, as the price is necessarily related to the class.

In [ ]:
print('Max Fare train:', train_data['Fare'].max())
print('-'*40)
print('Max Fare test', test_data['Fare'].max())

In [ ]:
# Determine the number of rows containing outliers
def outlier_fare_by_class(data, col, class_col):
    outliers_by_class = {}
        # Browse each class in 'Pclass' (1, 2, 3)
    for cls in data[class_col].unique():
        class_data = data[data[class_col] == cls]
        
        # Establish lower and upper quartiles
        q1 = class_data[col].quantile(0.25)
        q3 = class_data[col].quantile(0.75)

        # Determine the interquartile range
        iqr = q3 - q1

        # Calculate the outlier threshold value
        upper_bound = q3 + 1.5 * iqr

        # Find outlier by class
        outliers = class_data[(class_data[col] > upper_bound)]
        outliers_by_class[cls] = outliers
        
        # Display outlier information by class
        print(f"Class {cls} - Upper Outliers Threshold: {upper_bound}")
        print(f"Number of outliers in Class {cls}: {len(outliers)}")
    print('-'*40)
    

In [ ]:
print('Outlier for Fare in train_data :\n')
print(outlier_fare_by_class(train_data, 'Fare','Pclass'))
print('Outlier for Fare in test_data :\n')
print(outlier_fare_by_class(test_data, 'Fare','Pclass'))

In [ ]:
#Evaluate Parch and SibSp
names = ['Train', 'Test']
for data, name in zip(train_and_test, names):
    print(f"Max value of Parch in {name}:", data['Parch'].max())
    print(f"Max value of SibSp in {name}:", data['SibSp'].max())
    print('-'*40)

Large families were common in those days, and several parents or children could travel together.


#### Conclusion :
Certain types of models are more sensitive to outliers than others. When I get to the stage of building your model, consider whether to remove outliers for Fare, based on the type of model I decide to use.

## d) Visualization and feature engineering

We will now study the different variables using various visualizations to try to understand the correlations between them.
For visualizations, I don't think it's necessary to do this for train_data and test_data. It was relevant to check and correct the 2 datasets, but the 2 seem consistent with each other.

In [ ]:
# Variable survived
fig, ax = plt.subplots(2, 2, figsize=(22,15))

#Survived by sex
sns.countplot(data=train_data, x='Sex', hue='Survived', ax=ax[0,0])
ax[0,0].set_title("Survivors by gender", fontsize='20')
ax[0,0].set_xlabel("Gender", fontsize=12)
ax[0,0].legend(title="Survived", labels=["No", "Yes"])


# Survived by Class
sns.countplot(data=train_data, x='Pclass', hue='Survived', ax=ax[0,1])
ax[0,1].set_title("Survivors by Class", fontsize='20')
ax[0,1].set_xlabel("Class", fontsize=12)
ax[0,1].legend(title="Survived", labels=["No", "Yes"])

plt.tight_layout(pad=4)

#Survived by Age and Class
sns.violinplot(data=train_data, x='Pclass', y='Age', hue = 'Survived', split = True, ax=ax[1,0])
ax[1,0].set_title("Age distribution by class and survival rate", fontsize='20')
ax[1,0].legend(title="Survived", labels=["No", "Yes"], handles=[mpatches.Patch(color='#3174a1', label='No'), mpatches.Patch(color='#e1812b', label='Yes')])
ax[1,0].set_xlabel("Class", fontsize=12)

#Survived by Fare
bins = [0, 10, 20, 30, 50, 100, 200, 600]  
labels = ["0-10", "10-20", "20-30", "30-50", "50-100", "100-200", "200+"]
train_data['Price_range'] = pd.cut(train_data['Fare'], bins=bins, labels=labels,ordered=True)
sns.countplot(data=train_data, x='Price_range', hue='Survived', ax=ax[1,1])
ax[1,1].set_title("Survivors per ticket price", fontsize='20')
ax[1,1].set_xlabel("Ticket price", fontsize=12)
ax[1,1].legend(title="Survived", labels=["No", "Yes"])

#delete columns['Price_range']
train_data = train_data.drop(columns= "Price_range")

Logically, there are 3 determining factors that influence the survival rate:
- Money (with class and ticket price)
- Age
- Gender

The “women and children first” doctrine seems to apply to the sinking of the Titanic. For children, the third graph shows a higher survival rate. However, only the 3rd class appears to have dead infants.

This makes me wonder about the difference in survival rates between men and women in different classes.



In [ ]:
sns.catplot(data=train_data,x="Pclass",hue="Sex",col="Survived",kind="count",height=5, aspect = 1)

plt.subplots_adjust(top=0.9)
plt.suptitle("Survival Rate by Gender and Class")
plt.show()

The same applies to gender: 3rd class women have perished more than women in other classes.
“Women and children first” is not particularly true of the 3rd class.


We've highlighted the most obvious relationships.
We're now going to explore new features to find other correlations with survival.
The first, which I think is interesting and which we haven't yet explored, is family. Does being alone or in a family influence survival rates?


In [ ]:
#create columns 'Family size' and 'Alone'
def family(data) :
    data['FamilySize']= data['SibSp'] + data['Parch'] + 1
    data['Alone'] = (data['FamilySize'] == 1).astype(int)
    return data

train_and_test = [train_data, test_data]

for data in train_and_test :
    data = family(data)
    

fig, ax = plt.subplots(1, 2, figsize=(22,8))

# Survived by Family Size
sns.countplot(data=train_data, x='FamilySize', hue='Survived', ax=ax[0])
ax[0].set_title("Survivors by Family size", fontsize='20')
ax[0].set_xlabel("Family size", fontsize=12)
ax[0].legend(title="Survived", labels=["No", "Yes"])

# Survived Alone
sns.countplot(data=train_data, x='Alone', hue='Survived', ax=ax[1])
ax[1].set_title("Survivors solo traveler", fontsize='20')
ax[1].set_xlabel("Alone", fontsize=12)
ax[1].set_xticks(ticks=[0,1], labels=['In family', 'Alone'])
ax[1].legend(title="Survived", labels=["No", "Yes"])

#calculate survival_rate
survival_rate = train_data.groupby('Alone')['Survived'].mean()

print('Survival Rate :', survival_rate)

It would seem that living with a family can significantly improve survival rates. 

It may be easier for a family with children to board lifeboats than a single man.

We'll check this by separating the visualizations by gender.

In [ ]:
sns.barplot(data=train_data, x="Sex", y="Survived", hue="Alone")
plt.title('Survival rates by sex and status (alone or accompanied) aboard the Titanic')
plt.xlabel('Gender')
plt.ylabel('Survived (%)')
plt.legend(title="Alone",handles=[mpatches.Patch(color='#3174a1', label='No'), mpatches.Patch(color='#e1812b', label='Yes')])

Unsurprisingly, single men have a lower survival rate. The opposite is true for women. Several hypotheses can be put forward:
- Given that priority is given to women and children, not having children may have little impact on whether or not a woman gets into a lifeboat. It can even be a disadvantage: women on their own, not having to look after other people, could have boarded more quickly.
- Accompanied women may come from lower social classes, where survival rates were lower.
- Accompanied women (with children or partners) may have waited or hesitated to stay with their families.


Now I'd like to extract the titles from the 'Name' column.
I notice that the title seems to be made up in the same way: a capital letter followed by a period. It might be interesting to use this feature to extract it.

In [ ]:
#create column [Title]
def title(data) :
    data['Title'] = data['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
    return data

for data in train_and_test:
    data = title(data)

train_data['Title'].value_counts()

In [ ]:
#Separate Title
title_counts = train_data['Title'].value_counts()
titles_40plus = title_counts[title_counts >= 40].index
train_data_40plus = train_data[train_data['Title'].isin(titles_40plus)]

titles_40minus = title_counts[title_counts < 40].index
train_data_40minus = train_data[train_data['Title'].isin(titles_40minus)]

# Create Vizualisation
fig, ax = plt.subplots(1, 2, figsize=(22,8))

sns.countplot(data=train_data_40plus, x='Title', hue='Survived', ax=ax[0])
ax[0].set_title("Survivors by title", fontsize='20')
ax[0].legend(title="Survived", labels=["No", "Yes"])


sns.countplot(data=train_data_40minus, x='Title', hue='Survived', ax=ax[1])
ax[1].set_title("Survivors by title (Zoom for minus values)", fontsize='20')
ax[1].legend(title="Survived", labels=["No", "Yes"])
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation = 45)

plt.show()

Data by title corroborate previous analyses by gender and age : Miss and Master are unmarried women, often young, and teenagers.
Finally, the titles provide additional information on very few passengers. Statuses such as Dr, Lady, Sir are so poorly represented that it's impossible to draw any conclusions.


Now I'm going to look at the ticket variable. I'm going to see if each ticket is unique

In [ ]:
train_data['Ticket'].duplicated().sum()

To my surprise, there are many duplicate ticket numbers. Let's find out why.

In [ ]:

def groupsize(data) :
    data['GroupSize'] = data.groupby('Ticket')['Ticket'].transform('count')
    data[(data['GroupSize'] >1)]
    return data

for data in train_and_test :
    data = groupsize(data)

In [ ]:
print(train_data[(train_data['GroupSize'] > 1) & (train_data['Ticket'] == '113803')][['Name','Ticket']])
print('-' * 40)
print(train_data[(train_data['GroupSize'] > 1) & (train_data['Ticket'] == '349909')][['Name','Ticket']])
print('-' * 40)
print(train_data[(train_data['GroupSize'] > 1) & (train_data['Ticket'] == '347742')][['Name','Ticket']])


We can therefore deduce that the 'Fare' column is skewed.
Tickets are not issued individually, but as a family.
In my opinion, it's more than appropriate to create a 'Fare' column for each individual.

In [ ]:
# Create Fare per Person
train_data['Fare_Per_Person'] = train_data['Fare'] / train_data['GroupSize']
test_data['Fare_Per_Person'] = test_data['Fare'] / test_data['GroupSize']

There's only one variable left to analyze: 'Cabin'.
I'm aware that there are a large number of unknown values.
So I'm going to extract the first letter for each cabin entered, as from my research it seems that they correspond to the decks of the boat.

In [ ]:
# Extract first letter and replace missing values with 'U' (Unknow).
def CabinLetter(data) :
    data['CabinLetter'] = data['Cabin'].str[0]
    data['CabinLetter'] = data['Cabin'].fillna('U').str[0]
    return data

for data in train_and_test :
    data = CabinLetter(data)
    

print(train_data[['Cabin', 'CabinLetter']].head())

In [ ]:
#Explore Cabin Letter
sns.countplot(data=train_data, x='CabinLetter', hue='Survived')
plt.xlabel('Boat Deck')
plt.legend(title="Survived", labels=["No", "Yes"])
plt.title('Deck influence on survival', )

We can see that the cabin letter has an influence on the survival rate. Column U (Unknow) has the lowest survival rate.
By hypothesis, we could deduce that the lowest-cost tickets don't have a registered cabin, only the most affluent do.

In [ ]:
#Explore Cabin Letter
sns.countplot(data=train_data, x='CabinLetter', hue='Pclass')
plt.xlabel('Boat Deck')
plt.legend(title="Class")
plt.title('Class by deck', )

This proves that the cabins that are not indicated are mainly in 3rd class.
Given the proportions of missing values, I fear that grouping the 3 classes in the same U variable may induce a bias, given the over-representation of the 3rd class.
So we're going to look at various ways of being more precise.

My first idea is to combine the tickets, which represent a group, and see if the decks are missing from some and present in others. My idea is that, if we're traveling in a group, it seems logical to me that we should at least travel on the same deck. I could then replace the missing values with the values of the members of the other group.

In [ ]:
###### Train data

# Check if tickets have both “U” and a valid letter
ticket_groups = train_data.groupby('Ticket').agg(
    has_U=('CabinLetter', lambda x: (x == 'U').any()), 
    has_valid=('CabinLetter', lambda x: (x != 'U').any()) 
)

# Identify tickets with inconsistencies
ticket_groups['inconsistency'] = ticket_groups['has_U'] & ticket_groups['has_valid']

# Filter out inconsistent groups
inconsistent_tickets = ticket_groups[ticket_groups['inconsistency']].reset_index()

# Display results
print("Number of tickets with inconsistencies in train_data: ", len(inconsistent_tickets))
print(inconsistent_tickets)

print('-'*40)

###### Test data

# Check if tickets have both “U” and a valid letter
ticket_groups1 = test_data.groupby('Ticket').agg(
    has_U=('CabinLetter', lambda x: (x == 'U').any()), 
    has_valid=('CabinLetter', lambda x: (x != 'U').any()) 
)

# Identify tickets with inconsistencies
ticket_groups1['inconsistency'] = ticket_groups1['has_U'] & ticket_groups1['has_valid']

# Filter out inconsistent groups
inconsistent_tickets1 = ticket_groups1[ticket_groups1['inconsistency']].reset_index()

# Display results
print("Number of tickets with inconsistencies in test_data: ", len(inconsistent_tickets1))
print(inconsistent_tickets1)

In [ ]:
# Filter data with inconsistent tickets
inconsistent_data = train_data[train_data['Ticket'].isin(inconsistent_tickets['Ticket'])]
inconsistent_data_test = test_data[test_data['Ticket'].isin(inconsistent_tickets1['Ticket'])]

fig, ax = plt.subplots(1, 2, figsize=(22,8))

# View inconsistencies by ticket
sns.countplot(data=inconsistent_data, x='Ticket', hue='CabinLetter', ax=ax[0])
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation = 90)
ax[0].set_title('Cabin inconsistencies by ticket in train_data')
ax[0].set_xlabel("Ticket", fontsize=12)

sns.countplot(data=inconsistent_data_test, x='Ticket', hue='CabinLetter', ax=ax[1])
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation = 90)
ax[1].set_title('Cabin inconsistencies by ticket in test_data')
ax[1].set_xlabel("Ticket", fontsize=12)


Now that I've visualized this, I can replace all the U values with the known values of the other group members.

In [ ]:
# Create a function that replaces “U” with the known booth of the same ticket
def fill_cabin_from_group(group):
    # Check whether a known booth is present in the group
    known_cabin = group[group['CabinLetter'] != 'U']['CabinLetter'].unique()
    if len(known_cabin) > 0:  
        group['CabinLetter'] = group['CabinLetter'].replace('U', known_cabin[0])
    return group

# Apply function for each ticket group
train_data = train_data.groupby('Ticket').apply(fill_cabin_from_group).reset_index(drop=True)
test_data = test_data.groupby('Ticket').apply(fill_cabin_from_group).reset_index(drop=True)


I don't see any other way of finding the deck with the other data. So I'm going to classify the rest of the U's into 3:
U1 for Pclass1, U2 for Pclass2 and U3 for Pclass3.

In [ ]:
# Function to classify the remaining “U ”s according to class
def classify_remaining_U(row):
    if row['CabinLetter'] == 'U':
        return f"U{row['Pclass']}"
    return row['CabinLetter']

# Apply function to CabinLetter column
train_data['CabinLetter'] = train_data.apply(classify_remaining_U, axis=1)
test_data['CabinLetter'] = test_data.apply(classify_remaining_U, axis=1)
# Check results
print(train_data['CabinLetter'].value_counts())
print(''*40)
print(test_data['CabinLetter'].value_counts())

It looks pretty good, but there's just one value I'm not sure about: T. There's no T bridge, so it's probably an error.

In [ ]:
train_data[train_data['CabinLetter'] == 'T']

It's a class 1, so I'll change the value to U1.

In [ ]:
train_data['CabinLetter'] = train_data['CabinLetter'].replace('T', 'U1')

print(train_data['CabinLetter'].value_counts())

Now I'm going to clean the dataset again. Using a correlation map, I'm going to remove the least relevant columns following the creation of features. To do this, I'll also need to do some one-hot encoding for categorical data.

## e) Data cleaning following feature creation, column deletion and one-hot-encoding

Group the rarest titles into a 'Rare' category

In [ ]:
#Rare 'Title' category

def Title_rare(data) :
    data['Title'] = data['Title'].replace(['Lady', 'the Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    
train_and_test = [train_data, test_data]

for data in train_and_test :
    data = Title_rare(data)

train_data['Title'].value_counts()

I'm now going to purge the redundant columns.
For me, the columns to exclude are the following: 
- Name: Full name does not provide direct predictive information.
- PassengerId: Simple identifier, useless for prediction.
- Ticket: Contains unstructured information, difficult to use. I still used this data to extract FarePerPerson.
- Cabin: Too granular and often incomplete. CabinLetter is a good replacement.
- SibSp and Parch: Their information is already included in FamilySize and Alone.
- 'Fare' has been replaced by 'FarePerPerson', which is much more precise.

In [ ]:
#columns to drop
columns_to_drop = ['Name', 'Fare', 'Ticket', 'Cabin', 'SibSp', 'Parch']

# delete columns
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)


Check that the 2 datasets are ok.

In [ ]:
train_data.head()

In [ ]:
test_data.head()

Everything looks good. I'll now proceed with the one-hot-encoding.

In [ ]:
#columns to encode
columns_to_encode = ['Sex', 'Embarked', 'Title', 'CabinLetter']

#get dummies
train_data = pd.get_dummies(train_data, columns=columns_to_encode, drop_first=True, dtype=int)
test_data = pd.get_dummies(test_data, columns=columns_to_encode, drop_first=True, dtype=int)

train_data.head()

I'll check whether the train_data and test_data datasets have the same columns.

In [ ]:

missing_cols = set(train_data.columns) - set(test_data.columns)
print("Missing columns in test_data :", missing_cols)

Everything's ok, the 2 data sets are identical, except for “Survived”, which is normal given that this is the data to be predicted.Now let's create a heatmap to find out the correlations.

In [ ]:
train_data.columns

In [ ]:
#columns for correlation 
columns_for_correlation = ['Survived', 'Pclass', 'Age', 'Fare_Per_Person', 
                            'FamilySize', 'Sex_male', 'Embarked_Q', 
                            'Embarked_S', 'Title_Mr', 'Title_Mrs']

plt.figure(figsize=(16, 9))
heatmap = sns.heatmap(train_data[columns_for_correlation].corr(), vmin=-1, vmax=1, annot=True, cmap=sns.color_palette("vlag", as_cmap=True))
heatmap.set_title("Correlation Heatmap", fontsize=20)
plt.show()

The correlation map concludes with the same diagnosis: age, gender and class (including ticket price) seem to be most correlated with survival rate.

# 2) Model building
## a) Choose which models to test

I'm going to test 3 models: the Decision Tree, XGBoost and the Random Forest.
I chose these 3 models because they are less sensitive to the outliers that we were able to detect on Fare, and which are always present on FarePerPerson.

I separate the data data_train into train and test.
The data is unbalanced on the survival rate, so we will use the 'stratify' parameter.

In [ ]:
# Isolate the target variable (y)
y = train_data['Survived']

# Isolate the features (X)
X = train_data.drop(['Survived', 'PassengerId'], axis=1)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

## b) Decision Tree

At this stage we will begin the model optimisation process by constructing a decision tree model and set up cross-validated grid-search to exhuastively search for the best model parameters.
Additionally, we will also be setting our refit parameter to 'Accuracy',the metric used for the competition.

In [ ]:
# Instantiate the decision tree classifer
dt = DecisionTreeClassifier(random_state = 0)

# Create a dictionary of hyperparameters to tune
cv_params = {"max_depth": [2,4,6,8,10,15,20,30,40,50, None],
             "min_samples_leaf": [2,3,4,5,6,7,8,9, 10, 15, 20, 50],
             "min_samples_split": [2,3,4,5,6,7,8,9, 10, 15, 20, 50]}

# Define a set of scoring metrics to capture
scoring = {"accuracy", "precision", "recall", "f1", "roc_auc"}

# Instantiate the GridSearchCV object
clf_tree = GridSearchCV(dt,
                        cv_params,
                        scoring = scoring,
                        cv = 5,
                        refit = 'accuracy')

We will now fit the decision tree model to the training data.

In [ ]:
clf_tree.fit(X_train, y_train)

Examine the best Accuracy score achieved by the decision tree model on the training set.

In [ ]:
clf_tree.best_score_

Examine the best combination of hyperparameters

In [ ]:
# Determine the best combination of hyperparameters
clf_tree.best_params_

Next, we'll write a function called make_results() that will output all of the socres of our models. This function will accept three arguments.

In [ ]:
def make_results(model_name:str, model_object, metric:str):
    # Create dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {"accuracy": "mean_test_accuracy",
                   "precision": "mean_test_precision",
                   "recall": "mean_test_recall",
                   "f1": "mean_test_f1",
                   "roc_auc": "mean_test_roc_auc"}
    
    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)
    
    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]
    
    # Extract Accuracy, Precision, Recall, F1, and ROC_AUC socres from that row
    accuracy = best_estimator_results.mean_test_accuracy
    precision = best_estimator_results.mean_test_precision
    recall = best_estimator_results.mean_test_recall
    f1 = best_estimator_results.mean_test_f1
    roc_auc = best_estimator_results.mean_test_roc_auc
    
    # Create a table of results
    table = pd.DataFrame({"Model": [model_name],
                          "Accuracy": [accuracy],
                          "Precision": [precision],
                          "Recall": [recall],
                          "F1": [f1],
                          "ROC AUC": [roc_auc]
                         }
                        )
    
    return table

In [ ]:
# Create results table from the output of our Decision Tree Classifier object using our training data
results = make_results("Decision Tree CV", clf_tree, "accuracy")
results

Overall analysis :
- The model seems to perform well overall, with good accuracy and a good compromise between precision, recall and F1.
- Recall can still be improved, as a model performing better in recall would find more survivors, which could be important in certain contexts (for example, if the aim is to predict as many survivors as possible).
- The ROC AUC shows that your model has a good ability to separate classes, which is essential for binary classification problems.

## c) XGboost Model

Similar to the creation of our Decision Tree Model, to construct our XGBoost model, we will set up a cross-validated grid-search to iteratively search for the best model parameters.

In [ ]:
## Instantiate the XGBoost classifier
xgb = XGBClassifier(objective="binary:logistic", random_state=0)

# Create a dictionary of hyperparameters
cv_params = {"n_estimators": [100, 150, 200, 300],
    "learning_rate": [0.15, 0.2, 0.25],
    "max_depth": [4, 5, 6],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.6, 0.7, 0.8],
    "gamma": [0, 0.1, 0.5],
    "min_child_weight": [4, 5, 6],
    "reg_alpha": [0, 0.1, 0.2],
    "reg_lambda": [1, 1.5, 2]}
# Define a set of scoring metrics to capture
scoring = {"accuracy", "precision", "recall", "f1", "roc_auc"}

# Instantiate the GridSearchCV object
xgb_cv = GridSearchCV(xgb, cv_params, scoring=scoring, cv=5, refit='accuracy')


We will now fit the XGBoost model to the training data.

In [ ]:
%%time
xgb_cv = xgb_cv.fit(X_train, y_train)

Now we'll examine the best accuracy score achieved by the XGBoost model on the training dataset .

In [ ]:
xgb_cv.best_score_

Examine the best combination of hyperparameters

In [ ]:
# Determine the best combination of hyperparameters
xgb_cv.best_params_

In [ ]:
# Create results table from the output of our XGBoost Classifier object using our training data
xgb_results = make_results("XGBoost CV", xgb_cv, "accuracy")
xgb_results

## Overall analysis :
- The model correctly classified around 84.12% of the examples, which is a good score, indicating that the model performs well in all predictions.
- The model is relatively accurate when predicting the positive class (survival), with around 82% of its positive predictions being correct.
- The model has a good survivor detection rate with recall.
- The F1-Score, which balances precision and recall, indicates that the model has a good ability to predict both survivors and non-survivors.
- The AUC of 88.07% shows that the model separates classes well, with a high ability to discriminate between survivors and non-survivors.

## d) Random Forest Model

Similar to the creation of our Decision Tree Model and our XGBoost Model, to construct our Random Forest Model we will set up a cross-validated grid-search to iteratively search for the best model parameters.

In [ ]:
# Instantiate the random forest classifier
rf = RandomForestClassifier(random_state = 0)

# Create a dictionary of hyperparameters to tune
cv_params = {"n_estimators": [250, 300, 350],
    "max_depth": [8, 10, 12],
    "max_features": [0.6, 0.7, 0.8],
    "max_samples": [0.9, 1.0],
    "min_samples_leaf": [1, 2, 3],
    "min_samples_split": [4, 5, 6]}

# Define a set of scoring metrics to capture
scoring = {"accuracy", "precision", "recall", "f1", "roc_auc"}

# Instantiate the GridSearchCV object
rf_cv = GridSearchCV(rf, cv_params, scoring=scoring, cv=5, refit='accuracy')

We will now fit the random forest model to the training data.

In [ ]:
%%time
rf_cv.fit(X_train, y_train)

Now we'll examine the best accuracy score achieved by the random forest model on the training dataset .

In [ ]:
rf_cv.best_score_

Now we'll examine the best combination of hyperparameters.

In [ ]:
# Determine the best combination of hyperparameters
rf_cv.best_params_

## e) Choose the best model

We will combine the evaluation scores on the training set for both the decision tree, XGBoost and random forest models.

In [ ]:
# Get scores on the training data for the random forest model and add this to the existing results table
rf_results = make_results("Random Forest CV", rf_cv, "accuracy")
combined_results = pd.concat([results, rf_results,xgb_results ], axis=0)
combined_results

We chose XGBoost for this project because all the scores were higher or equal.
What's more, we're looking for the best accuracy, and XGBoost has the best result. 
So it seems obvious to me to choose this model.

Now that we have chosen our model (XGBoost), we can evaluate this model on the test set. First we will use our model to predict on the test data and assign the results to a variable called rf_preds.

In [ ]:
# Get scores on test data
xgb_preds = xgb_cv.best_estimator_.predict(X_test)

Use the following get_test_scores() function to obtain the scores of our model on the test data.

In [ ]:
def get_test_scores(model_name:str, preds, y_test_data):    
    # Extract Accuracy, Precision, Recall, F1, and ROC AUC scores
    accuracy = metrics.accuracy_score(y_test_data, preds)
    precision = metrics.precision_score(y_test_data, preds)
    recall = metrics.recall_score(y_test_data, preds)
    f1 = metrics.f1_score(y_test_data, preds)
    roc_auc = metrics.roc_auc_score(y_test_data, preds)
    
    # Create a table of results
    table = pd.DataFrame({"Model": [model_name],
                          "Accuracy": [accuracy],
                          "Precision": [precision],
                          "Recall": [recall],
                          "F1": [f1],
                          "ROC AUC": [roc_auc]
                         }
                        )
    
    return table

We'll now use the get_test_scores() function to generate the scores on the test data and assign the results to xgb_test_scores. We will concat these results to our combined table for comparison.

In [ ]:
# Get scores on test data
xgb_test_scores = get_test_scores("XGB Test", xgb_preds, y_test)
combined_results = pd.concat([combined_results, xgb_test_scores], axis=0)
combined_results

Compared with cross-validation (XGBoost CV), we see a slight drop in performance on the test set, notably a reduction in Accuracy and ROC AUC. This indicates that XGBoost may present a slight over-fit to the training data, which is not uncommon with powerful models like this one.



## f) Feature importance

By way of illustration, I'm now going to look at the features importances of the decision tree and random forest models.

#### Decision Tree

In [ ]:
#plot the decision tree
plt.figure(figsize=(70,24))
plot_tree(clf_tree.best_estimator_, max_depth=6, fontsize=10, feature_names=X.columns,
          class_names={0:"Dead", 1:"Survived"}, filled=True)
plt.show()

In [ ]:
# Calculate the important features from the featured engineered decision tree model and save into a dataframe
importances = pd.DataFrame(clf_tree.best_estimator_.feature_importances_,
                           columns=['gini_importance'],
                           index=X.columns)

# Sort the 'importances' dataframe in descending order
importances = importances.sort_values(by='gini_importance', ascending=False)

# Only extract the features with importances > 0
importances = importances[importances['gini_importance'] > 0]
importances

In [ ]:
# Create a barplot to visualise the decision tree feature importances
sns.barplot(data=importances, x='gini_importance', y=importances.index, orient='h')
plt.title("Decision Tree: Feature Importances for Titanic Disaster", fontsize=14)
plt.ylabel("Feature")
plt.xlabel("Importance")
plt.show()

Conclusion from the decision tree: 
The most important feature is mainly gender, as we suspected.
Next comes social class with Fare Per person; and Age.
Group size is more important than family size.

#### XGBoost

We'll now repeat the same process as previously undertaken to determine the feature importances for the random forest model

In [ ]:
# Calculate the important features from the featured engineered random forest model and save to a new dataframe
xgb_importances = pd.DataFrame(xgb_cv.best_estimator_.feature_importances_,
                              columns=['gini_importance'],
                              index=X.columns)

# Sort the 'rf_importances' dataframe in descending order
xgb_importances = xgb_importances.sort_values(by='gini_importance', ascending=False)

# Only extract the features with importances > 0
xgb_importances = xgb_importances[xgb_importances['gini_importance'] > 0]
xgb_importances

In [ ]:
plt.figure(figsize=(12,7))
sns.barplot(data=xgb_importances, x='gini_importance', y=xgb_importances.index, orient='h')
plt.title("XGBoost : Feature Importances for Titanic disaster", fontsize=14)
plt.ylabel("Feature")
plt.xlabel("Importance")
plt.show()

Conclusion for XGBoost model : 

As with the Decision Tree, gender is really decisive.
Here, Cabin Letter U3, which represents class 3, is of great importance. And the fare per person seems to be less important than with the Decision Tree.

#### Random Forest

We'll now repeat the same process as previously undertaken to determine the feature importances for the random forest model.

In [ ]:
# Calculate the important features from the featured engineered random forest model and save to a new dataframe
rf_importances = pd.DataFrame(rf_cv.best_estimator_.feature_importances_,
                              columns=['gini_importance'],
                              index=X.columns)

# Sort the 'rf_importances' dataframe in descending order
rf_importances = rf_importances.sort_values(by='gini_importance', ascending=False)

# Only extract the features with importances > 0
rf_importances = rf_importances[rf_importances['gini_importance'] > 0]
rf_importances

In [ ]:
# Create a barplot to visualise the random forest feature importances
plt.figure(figsize=(12,7))
sns.barplot(data=rf_importances, x='gini_importance', y=rf_importances.index, orient='h')
plt.title("Random Forest: Feature Importances for Titanic Disaster", fontsize=14)
plt.ylabel("Feature")
plt.xlabel("Importance")
plt.show()

Conclusion for Random Forest :

The Random Forest model strongly resembles the Decision Tree, where gender and Fare per person seem to be the determining factors.


# 3) Submission and Conclusion
## a) Generate file for submission

I prepare the test data by deleting PassengerID.

In [ ]:
# Save PassengerID for submission
test_ids = test_data["PassengerId"]

#Delete for X_test
X_test = test_data.drop(columns=["PassengerId"])  

In [ ]:
#Test dataset predictions
predictions = xgb_cv.best_estimator_.predict(X_test)

#Creating a submission file
submission = pd.DataFrame({"PassengerId": test_ids, "Survived": predictions})

submission.to_csv("submission.csv", index=False)
print("Submission file created : submission.csv")

## b) Conclusion

What we can learn from this study is that the determining factors were mainly gender, social class and age.
However, a closer look at the data reveals the complexity involved in predicting survival on a shipwreck such as this.
Other details, which may seem insignificant (such as family size) can ultimately be decisive.

I'd like to thank all those who have read this. This is my first Kaggle/ contest.
Don't hesitate to vote positively if you liked it.